# Chapter 5: 오차역전파법

> ### **5.1** 계산 그래프
> * 계산 그래프: 계산 과정을 그래프로 나타낸 것  
> (복수의 노드와 엣지로 표현)  
>
> * 계산 그래프 문제 풀이 흐름
>   1. 계산 그래프를 구성한다.
>   2. 그래프에서 계산을 왼쪽에서 오른쪽으로 진행한다  
>     (순전파, 그래프의 출발점부터 종착점으로의 전파)  
>
> * 역전파 (backward propagation): 종참점에서 출발점으로의 전파
>
> ![cal_graph](cal_graph.png)
> ![cal_graph](cal_graph_r.png)
> ![cal_graph](cal_graph_2.png)


>> ##### 국소적 계산
>> * 국소적: 자신과 직접 관계된 작은 범위  
>>
>> ![partial_cal](partial_cal.png)

>> ##### 왜 계산 그래프를 활용하는가?
>> * 국소적 계산  
>>      * 전체가 아무리 복잡해도 각 노드에서는 단순한 계산에 집중하여 문제를 단순화할 수 있음
>> * 중간 계산 결과를 모두 보관할 수 있음
>> * 역전파를 통해 미분을 효율적으로 계산할 수 있음
>>      * 문제 1은 사과를 2개 사서 소비세를 포함한 최종 금액을 구하는 문제였음   
>>        여기서 사과 가격이 오르면 최종 금액에 어떤 영향을 끼치는지 구해보면, 이는 '사과 가격에 대한 지불 금액의 미분'을 구하는 문제에 해당  
>>        이 미분 값은 사과 값이 '아주 조금' 올랐을 때, 지불 금액이 얼마나 증가하느냐를 표시한 것
>>
>> ![cal_graph_back](cal_graph_back.png)
>>
>> * 예시에서는 사과 가격에 대한 미분만 구했지만, '소비세에 대한 지불 금액의 미분'이나 '사과 개수에 대한 지불 금액의 미분'도 같은 순서로 구할 수 있음  
>> * 이러한 과정에서 중간 과정까지 구한 미분 결과를 공유할 수 있어, 다수의 미분을 효율적으로 계산할 수 있음

> ### **5.2** 연쇄법칙
> * 역전파는 '국소적인 미분'을 순방향과는 반대인 종착점에서 출발점으로 전달함
> * 이 때에 '국소적 미분'을 전달하는 원리는 연쇄법칙에 따른 것
>
>> ##### 계산 그래프의 역전파
>> ![back_cal_graph](back_cal_graph.png)
>>
>> ##### 연쇄법칙이란
>> * 합성 함수: 여러 함수로 구성된 함수  
>>
>> ![function_composition](function_composition.png)
>> ##### $\frac{\sigma z}{\sigma x} (x$ 에 대한 $z$의 미분 $)$은 $\frac{\sigma z}{\sigma t} (t$ 에 대한 $z$의 미분 $)$과 $\frac{\sigma t}{\sigma x} (x$ 에 대한 $t$의 미분 $)$의 곱으로 나타낼 수 있음
>> ![expressions](expressions.png)
>>
>> ##### 연쇄법칙과 계산 그래프
>> ![backward_graph](backward_graph.png)
>> ![연쇄법칙_backward](연쇄법칙_backward.png)

> ### **5.3** 역전파
>
>> ##### 덧셈 노드의 역전파
>>
>> ![plus_node](plus_node.png)
>> ![picture_5-10](picture_5-10.png)
>> ![plus_node_example](plus_node_example.png)

>> ##### 곱셈 노드의 역전파
>>
>> ![backward_product](backward_product_1.png)
>> ![product_node_ex](product_node_ex.png)
>> ![product_node_explain](product_node_explain.png)

>> ##### 사과 쇼핑의 예
>> * 사과의 가격, 사과의 개수, 소비세라는 세 변수 각각이 최종 금액에 어떻게 영향을 주느냐를 푸는 문제  
>> &rarr; 사과 가격에 대한 지불 금액의 미분  
>> &rarr; 사과 개수에 대한 지불 금액의 미분  
>> &rarr; 소비세에 대한 지불 금액의 미분
>>
>> ![apple_shopping](apple_shopping.png)

> ### **5.4** 단순한 계층 구현하기
>
>> ##### 곱셈 계층
>> * 모든 계층은 forward()와 backward()라는 공통 method를 갖도록 구현  
>>   &rarr; forward(): 순전파  
>>   &rarr; backward(): 역전파

In [56]:
# 곱셈 계층
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x*y

        return out
    
    def backward(self, dout):
        dx = dout*self.y
        dy = dout*self.x

        return dx, dy

>> __init__()에서는 인스턴스 변수인 x와 y를 초기화  
>>   &rarr; x,y: 순전파 시의 입력 값을 유지하기 위해 사용  
>> forward()에서는 x와 y를 인수로 받고 두 값을 곱해서 반환  
>> backward()에서는 상류에서 넘어온 미분(dout)에 순전파 때의 값을 서로 바꿔 곱한 후 하류로 흘려보냄
>>  
>> ![mullayer_apple_shopping](mullayer_apple_shopping.png)

In [57]:
apple = 100
apple_num = 2
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

# 역전파
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

220.00000000000003
2.2 110.00000000000001 200


>> ##### 덧셈 계층

In [58]:
class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x+y
        return out
    
    def backward(self, dout):
        dx = dout*1
        dy = dout*1
        return dx, dy

>> 덧셈 계층에서는 초기화가 필요 없으니 __init__()에서 아무 일도 하지 않음
>>   &rarr; pass: 아무것도 하지 말라는 명령
>> forward()에서는 입력받은 두 인수 x, y를 더해서 반환
>> backward()에서는 상류에서 내려온 미분(dout)을 그대로 하류로 흘림
>>  
>> ![addlayer_apple_shopping](addlayer_apple_shopping.png)

In [59]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

# 역전파
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(price)
print(dapple_num, dapple, dorange, dorange_num, dtax)

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


> ### **5.5** 활성화 함수 계층 구현하기
>
>> ##### ReLU 계층
>> ![relu_graph](relu_graph.png)

In [60]:
# ReLU layer
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x<=0)
        out = x.copy()
        out[self.mask] = 0

        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx

>> Relu 클래스는 mask라는 인스턴스 변수를 가짐  
>>   &rarr; mask: True/False로 구성된 numpy array, 순전파의 입력인 x의 원소 값이 0 이하인 인덱스는 True, 그 외(0보다 큰 원소)는 False로 유지
>>  
>> 그림 5-18과 같이 순전파 때의 입력 값이 0 이하면 역전파 때의 값은 0이 돼야 함  
>> &rarr; 역전파 때는 순전파 때 만들어둔 mask를 써서 mask의 원소가 True인 곳에는 상류에서 전파된 dout을 0으로 설정

>> ##### Sigmoid 계층
>>  
>> ![sigmoid_forward](sigmoid_forward.png)
>> ![sigmoid_backward_1](sigmoid_backward_1.png)
>> ![sigmoid_backward_1_g](sigmoid_backward_1_g.png)
>> ![sigmoid_backward_1](sigmoid_backward_1.png)
>> ![sigmoid_backward_2](sigmoid_backward_2.png)
>> ![sigmoid_backward_3](sigmoid_backward_3.png)
>> ![sigmoid_backward_4](sigmoid_backward_4.png)
>>
>> $ \frac{\sigma L}{\sigma y} y^2 \exp(-x) $은 순전파의 입력 $x$와 출력 $y$만으로 계산할 수 있음  
>>   &rarr; 그림 5-20의 계산 그래프ㅡ이 중간 과정을 모두 묶어 단순한 sigmoid 노드 하나로 대체할 수 있음  
>>  
>> ![sigmoid_node](sigmoid_node.png)
>>
>> sigmoid 노드로 간소화 하면 역전파 과정의 중간 계산들을 생략할 수 있어 더 효율적임
>> 노드를 그룹화하여 sigmoid 계층의 세세한 내용을 노출하지 않고 입력과 출력에만 집중할 수 있음
>>  
>> ![sigmoid_cal](sigmoid_cal.png)
>> ![sigmoid_node_another](sigmoid_node_another.png)


In [61]:
# Sigmoid layer
import numpy as np

class Sigmoid:
    def __init__(self):
        self.out = None
    
    def forward(self, x):
        out = 1/(1+np.exp(-x))
        self.out = out
    
    def backward(self, dout):
        dx = dout*(1.0-self.out)*self.out

        return dx

>> 위 구현에서 순전파의 출력을 인스턴수 변수 out에 보관했다가, 역전파 계산 때 그 값을 사용

> ### **5.6** Affine/Softmax 계층 구현하기
>>  
>> ##### Affine 계층
>> * 신경망의 순전파에서는 가중치 신호의 총합을 계산하기 때문에 행렬의 곱을 사용 (3.3 다차원 배열의 계산 참고)

In [62]:
X = np.random.rand(2)   # input
W = np.random.rand(2,3) # weight
B = np.random.rand(3)   # bias

print(X.shape)
print(W.shape)
print(B.shape)

Y = np.dot(X, W)+B

(2,)
(2, 3)
(3,)


>> 위 구현에서 뉴런의 가중치 합은 `Y = np.dot(X, W)+B`처럼 계산  
>> 이를 활성화 함수로 변환해 다음 층으로 전파하는 것이 신경망 순전파의 흐름  
>> 행렬의 곱 계산은 대응하는 차원의 원소 수를 일치시켜야 함  
>>  
>> ![matrix_product_affine](matrix_product_affine.png)
>>  
>> 이를 계산 그래프로 표현하면
>>  
>> ![matrix_cal_graph](matrix_cal_graph.png)  
>> ![matrix_backward](matrix_backward.png)
>> ![matrix_backward_g](matrix_backward_g.png)
>>  
>> * 행렬의 형상에 주의해야 하는 이유  
>>      &rarr; 행렬의 곱에서는 대응하는 차원의 원소 수를 일치시켜야 함
>>      &rarr; 이를 위해서는 식 5.13을 동원해야 할 수도 있기 때문
>>  
>> ![matrix_product_backward](matrix_product_backward.png)

>> ##### 배치용 Affine 계층
>> * 지금까지 살펴본 affine 계층은 입력 데이터로 X 하나만을 고려한 것
>>  
>> ![batch_affine](batch_affine.png)

In [63]:
X_dot_W = np.array([[0,0,0], [10,10,10]])
B = np.array([1,2,3])

print(X_dot_W)
print(X_dot_W+B)

[[ 0  0  0]
 [10 10 10]]
[[ 1  2  3]
 [11 12 13]]


>> 순전파의 편향 덧셈은 각각의 데이터(1번째 데이터, 2번째 데이터, ...)에 더해짐  
>> &nbsp; &nbsp; &rarr; 역전파 때는 각 데이터의 역전파 값이 편향의 원소에 모여야 함

In [64]:
dY = np.array([[1,2,3], [4,5,6]])
print(dY)

dB = np.sum(dY, axis=0)
print(dB)

[[1 2 3]
 [4 5 6]]
[5 7 9]


>> 이 예에서는 데이터가 2개(N=2)라고 가정, 편향의 역전파는 그 두 데이터에 대한 미분을 데이터마다 더해서 구함  
>> &nbsp; &nbsp; &rarr; np.sum()에서 0번째 축(데이터를 단위로 한 축)에 대해서 (axis=0)의 총합을 구하는 것

In [65]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
    
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W)+self.b
        
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis = 0)

        return dx

>> ##### Softmax-with-Loss 계층
>>  
>> ![affine_relu_softmax_layer](affine_relu_softmax_layer.png)
>> ![softmax_with_loss_g](softmax_with_loss_g.png)
>> ![softmax-with-loss_cal_g](softmax-with-loss_cal_g.png)
>>  
>> 위 그래프는 3클래스 분류를 수행하는 신경망 가정
>> * 이전 계층으로부터의 입력: (a1, a2, a3)
>> * Softmax 계층 출력: (y1, y2, y3)
>> * 정답 레이블: (t1, t2, t3)
>> * Cross Entropy Error 게층 출력: 손실 L
>> * Softmax-with-Loss 계층의 역전파 결과: (y1-t1, y2-t2, y3-t3)

>>> ##### 순전파
>>>  
>>> ![softmax_layer](softmax_layer.png)  
>>> 위 그래프에서는 지수의 합, 즉 식 A.1의 분모 항을 S로 표기, 최종 출력은 (y1, y2, y3)
>>>  
>>> ![cross_entropy_error_cal_g](cross_entropy_error_cal_g.png)


>>> ##### 역전파
>>> ![cee_backward](cee_backward.png)
>>>  
>>> 위 계산 그래프의 역전파를 구할 때 규칙(주의 사항)
>>> * 역전파의 초깃값, 즉 그림 A-4의 가장 오른쪽 역전파의 값은 1
>>> * 'X'노드의 역전파는 순전파 시의 입력들의 값을 '서로 바꿔' 상류의 미분에 곱하고 하류로 흘림
>>> * '+' 노드에서는 상류에서 전해지는 미분을 그대로 흘림
>>> * 'log' 노드의 역전파는 다음 식을 따름  
>>> &nbsp; &nbsp; &rarr; $ y=\log{}{x} $  
>>> &nbsp; &nbsp; &nbsp; &nbsp; $ \frac{\sigma y}{\sigma x}=\frac{1}{x} $
>>>  
>>> 위의 규칙을 따르면 Cross Entropy Error 계층의 역전파는 쉽게 구할 수 있음  
>>> 결과는 $ (- \frac{t1}{y1}, - \frac{t2}{y2}, - \frac{t3}{y3}) $ 이며, 이 값이 Softmax 계층의 역전파 입력이 됨  
>>>  
>>> ![softmax_backward_1](softmax_backward_1.png)
>>> ![softmax_backward_2](softmax_backward_2.png)
>>> ![softmax_backward_3](softmax_backward_3.png)
>>> ![softmax_backward_4](softmax_backward_4.png)
>>> ![softmax_backward_5](softmax_backward_5.png)
>>> ![softmax_backward_6](softmax_backward_6.png)
>>> ![softmax_with_loss](softmax_with_loss.png)
>>>  
>>> Softmax-with-Loss에서 주목할 것은 역전파의 결과  
>>> * softmax 계층의 역전파는 (y1-t1, y2-t2, y3-t3)로, softmax 계층의 출력과 정답 레이블의 차분임  
>>> * 신경망의 역전파에서는 이 차이인 오차가 앞 계층에 전해짐  
>>> &nbsp; &nbsp; &rarr; 신경망 학습의 중요한 성질  
>>>  
>>> 신경망 학습의 목적은 신경망의 출력(Softmax의 출력)이 정답 레이블과 가까워지도록 가중치 매개변수의 값을 조정하는 것  
>>> &nbsp; &nbsp; &rarr; 신경망의 출력과 정답 레이블의 오차를 효율적으로 앞 계층에 전달해야 함  
>>> &nbsp; &nbsp; &rarr; (y1-t1, y2-t2, y3-t3)라는 결과는 바로 Softmax 계층의 출력과 정답 레이블의 차이로, 신경망의 현재 출력과 정답 레이블의 오차를 있는 그래도 드러냄
>>>  
>>> ![backward_note](backward_note.png)
>>>  
>>> 구체적인 예
>>> * 정답 레이블: (0, 1, 0)
>>> * Softmax 계층 출력: (0.3, 0.2, 0.5)
>>> * 정답 레이블을 보면 정담의 인덱스는 1, 그런데 출력에서는 이때의 확률이 0.2로 신경망이 제대로 인식하지 못하고 있음
>>> * 이 때 Softmax 계층의 역전파는 (0.3, -0.8, 05)의 오차를 전파함


In [66]:
# softmax function, complement overflow
def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a-c)
    sum_exp_a = np.sum(exp_a)
    y = exp_a/sum_exp_a
    
    return y

# CEE, batch, one-hot label
def cross_entropy_error(y,t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshpae(1, y.size)

    batch_size = y.shape[0]
    
    return -np.sum(t*np.log(y+1e-7))/batch_size

class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None    # 손실
        self.y = None   # softmax의 출력
        self.t = None   # 정답 레이블 (one-hot 벡터)

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y-self.t)/batch_size

        return dx

> ### **5.7** 오차역전파법 구현하기
>>  
>> ##### 신경망 학습의 전체 그림
>> &nbsp; 1 단계 - 미니배치  
>> &nbsp; &nbsp; 훈련 데이터 중 일부를 무작위로 가져옴, 이렇게 선별한 데이터를 미니배치라 하며 미니배치의 손실함수 값을 줄이는 것이 목표  
>> &nbsp; 2단계 - 기울기 산출  
>> &nbsp; &nbsp; 미니배치의 손실 함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구함, 기울기는 손실 함수의 값을 가장 작게 하는 방향 제시  
>> &nbsp; 3단계 - 매개변수 갱신  
>> &nbsp; &nbsp; 가중치 매개변수를 기울기 방향으로 아주 조금 갱신  
>> &nbsp; 4단계 - 반복  
>> &nbsp; &nbsp; 1~3단계 반복  
>>  
>> 오차역전파법이 등장하는 단계는 기울기 산출 단계
>> * 이전에는 기울기를 구하기 위해 수치 미분을 사용  
>> &nbsp; &nbsp; &rarr; 수치 미분은 구현하기 쉽지만 계산이 오래걸림  
>> &nbsp; &nbsp; &rarr; 오차역전파법을 이용하면 느린 수치 미분과 달리 기울기를 효율적으로, 빠르게 구할 수 있음  
>>  
>> ##### 오차역전파법을 적용한 신경망 구현하기
>> * TowLayerNet 클래스로 구현
>>  
>> ![2layer_variable_methods](2layer_variable_methods.png)

In [67]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

class TowLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std*np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std*np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] =  Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.lastLayer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        
        if t.ndim!=1 : t=np.argmax(t, axis=1)

        accuracy = np.sum(y==t)/float(x.shape[0])

        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
    
    def gradient(self, x, t):
        # 순전파
        self.loss(x, t)

        # 역전파
        dout = 1
        dout = self.lastLayer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        
        for layer in layers:
            dout = layer.backward(dout)
        
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db

        return grads


>> 신경망의 계층이 OrderedDict에 보관되어 있음  
>> &nbsp; &nbsp; &rarr; 순전파 때는 추가한 순서대로 각 계층의 forward() method를 호출하기만 하면 처리 완료  
>> &nbsp; &nbsp; &rarr; 역전파 때는 계층을 반댈 순서로 호출하기만 하면 처리 완료
>>  
>> 이처럼 신경망의 구성 요소를 계층으로 구현한 덕분에 신경망을 쉽게 구축 가능

>> ##### 오차역전파법으로 구한 기울기 검증하기
>> * 수치 미분은 느리고 오차역전파법을 제대로 구현해두면 더 이상 필요 없음  
>> &nbsp; &nbsp; &rarr; 수치 미분은 오차역전파법을 구현했는지 확인하기 위해 활용
>> * 수치 미분의 이점은 구현하기 쉽다는 것  &rarr;  수치 미분의 구현에는 버그가 숨어있기 어려움  
>> &nbsp; &nbsp; &rarr; 수치 미분의 결과와 오차역전파법의 결과를 비교하여 오차역전파법을 제대로 구현했는지 검증함  
>> &nbsp; &nbsp; &rarr; 이러한 과정을 '기울기 확인' 이라고 함

In [68]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TowLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 각 가중치의 차이의 절댓값을 구한 후, 그 절댓값들의 평균을 낸다\
for key in grad_numerical.keys():
    diff = np.average(np.abs(grad_backprop[key]-grad_numerical[key]))
    print(key+': '+str(diff))

W1: 4.192481287058076e-10
b1: 2.6031925072077677e-09
W2: 5.907065699895281e-09
b2: 1.3946492104371933e-07


>> 위 결과를 보면 수치 미분과 오차역전파법으로 구한 기울기의 차가 매우 작다는 것을 말해줌

>> ##### 오차역전법을 사용한 학습 구현하기

In [69]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TowLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size/batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # 오차역전파법으로 기울기를 구한다
    grad = network.gradient(x_batch, t_batch)

    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate*grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)


0.1082 0.1126
0.90275 0.9042
0.9228166666666666 0.9227
0.9377333333333333 0.9367
0.9457 0.945
0.9515 0.9496
0.9570666666666666 0.9554
0.9612333333333334 0.9583
0.9639666666666666 0.9602
0.9663166666666667 0.9624
0.9699833333333333 0.963
0.9718833333333333 0.9669
0.9731 0.9671
0.97375 0.9673
0.9757666666666667 0.968
0.9781166666666666 0.9697
0.9779166666666667 0.9703
